## Step 1: Initialize Spark Session

Create Spark session. Configuration is automatically loaded from `spark-defaults.conf`.

# OLake Weather Data Analysis with Apache Spark

This notebook demonstrates querying Iceberg tables using Apache Spark after data has been synced from MySQL to Iceberg via OLake.

## Configuration

All Spark and Iceberg configurations are loaded from mounted configuration files:
- `/opt/spark/conf/spark-defaults.conf` - Spark and Iceberg settings
- `/opt/spark/conf/core-site.xml` - Hadoop S3A configuration  
- `/opt/spark/conf/catalog/iceberg.properties` - Iceberg REST catalog settings

In [1]:
from pyspark.sql import SparkSession
import os

# Verify configuration files are mounted
print("Configuration files:")
print(f"  SPARK_CONF_DIR: {os.getenv('SPARK_CONF_DIR')}")
print(f"  HADOOP_CONF_DIR: {os.getenv('HADOOP_CONF_DIR')}")

# Create Spark session
spark = SparkSession.builder \
    .appName("OLake Weather Analysis") \
    .config("spark.hadoop.fs.s3.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .getOrCreate()

print("\n✅ Spark session initialized successfully!")
print(f"Spark version: {spark.version}")

Configuration files:
  SPARK_CONF_DIR: /opt/spark/conf
  HADOOP_CONF_DIR: /opt/spark/conf

✅ Spark session initialized successfully!
Spark version: 3.5.0


## Step 2: Verify Configuration

Check that Iceberg catalog configuration was loaded correctly.

In [2]:
print("Loaded Iceberg Configuration:")
print("="*60)
for key, value in spark.sparkContext.getConf().getAll():
    if 'iceberg' in key.lower() or 's3' in key.lower():
        # Mask sensitive values
        if 'secret' in key.lower() or 'password' in key.lower():
            value = '***MASKED***'
        print(f"{key}: {value}")

Loaded Iceberg Configuration:
spark.hadoop.fs.s3a.connection.ssl.enabled: false
spark.hadoop.fs.s3a.path.style.access: true
spark.hadoop.fs.s3a.endpoint: http://minio:9090
spark.hadoop.fs.s3a.aws.credentials.provider: org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider
spark.sql.catalog.iceberg.warehouse: s3a://warehouse/
spark.hadoop.fs.s3.impl: org.apache.hadoop.fs.s3a.S3AFileSystem
spark.sql.catalog.iceberg.io-impl: org.apache.iceberg.hadoop.HadoopFileIO
spark.hadoop.fs.s3a.access.key: minio
spark.sql.catalog.iceberg.type: rest
spark.hadoop.fs.s3a.impl: org.apache.hadoop.fs.s3a.S3AFileSystem
spark.sql.catalog.iceberg: org.apache.iceberg.spark.SparkCatalog
spark.sql.catalog.iceberg.uri: http://rest:8181
spark.hadoop.fs.s3a.secret.key: ***MASKED***


## Step 3: Verify Iceberg Catalog Connection

Check available catalogs and namespaces.

In [3]:
print("Available Catalogs:")
spark.sql("SHOW CATALOGS").show()

print("\nNamespaces in Iceberg Catalog:")
spark.sql("SHOW NAMESPACES IN iceberg").show()

Available Catalogs:
+-------------+
|      catalog|
+-------------+
|spark_catalog|
+-------------+


Namespaces in Iceberg Catalog:
+--------------------+
|           namespace|
+--------------------+
|          test_olake|
|weather_sync_weather|
+--------------------+



## Step 4: List Tables in Weather Database

After running the OLake sync job, you should see the `weather` table here.

In [4]:
# List tables in weather namespace
print("Tables in weather namespace:")
spark.sql("SHOW TABLES IN iceberg.weather_sync_weather").show()

Tables in weather namespace:
+--------------------+---------+-----------+
|           namespace|tableName|isTemporary|
+--------------------+---------+-----------+
|weather_sync_weather|  weather|      false|
+--------------------+---------+-----------+



## Step 5: Query Weather Data

Now let's query the actual weather data synced from MySQL!

In [15]:
# Read the weather table
weather_df = spark.table("iceberg.weather_sync_weather.weather")

print(f"Total records: {weather_df.count():,}")
print("\nFirst 10 rows:")
weather_df.show(10, truncate=False)

Total records: 33,486

First 10 rows:
+---------------+----------+---------------+-------------+-----------------------+-------------+------------+--------------+------------+--------+----------------+-----+---------+----------+---------+----------+-------------------+------------+---------------+
|temperature_avg|date_month|temperature_min|station_state|_olake_timestamp       |precipitation|date_week_of|wind_direction|station_city|_op_type|station_location|id   |date_year|date_full |_olake_id|wind_speed|_cdc_timestamp     |station_code|temperature_max|
+---------------+----------+---------------+-------------+-----------------------+-------------+------------+--------------+------------+--------+----------------+-----+---------+----------+---------+----------+-------------------+------------+---------------+
|39.0           |1         |32.0           |Alabama      |2025-11-19 20:35:46.381|0.0          |3           |33            |Birmingham  |c       |Birmingham, AL  |83716|2016     |

In [16]:
# Check the schema
print("Weather table schema:")
weather_df.printSchema()

Weather table schema:
root
 |-- temperature_avg: float (nullable = true)
 |-- date_month: integer (nullable = true)
 |-- temperature_min: float (nullable = true)
 |-- station_state: string (nullable = true)
 |-- _olake_timestamp: timestamp (nullable = true)
 |-- precipitation: float (nullable = true)
 |-- date_week_of: integer (nullable = true)
 |-- wind_direction: integer (nullable = true)
 |-- station_city: string (nullable = true)
 |-- _op_type: string (nullable = true)
 |-- station_location: string (nullable = true)
 |-- id: integer (nullable = true)
 |-- date_year: integer (nullable = true)
 |-- date_full: string (nullable = true)
 |-- _olake_id: string (nullable = false)
 |-- wind_speed: float (nullable = true)
 |-- _cdc_timestamp: timestamp (nullable = true)
 |-- station_code: string (nullable = true)
 |-- temperature_max: float (nullable = true)



## Step 6: Data Analysis Examples

Perform various analytics on the weather data using Spark SQL.

In [8]:
# Average temperature by state
print("Average Temperature by State (Top 10):")
avg_temp_by_state = spark.sql("""
    SELECT 
        station_state,
        ROUND(AVG(temperature_avg), 2) as avg_temp,
        ROUND(AVG(temperature_max), 2) as avg_max_temp,
        ROUND(AVG(temperature_min), 2) as avg_min_temp,
        COUNT(*) as record_count
    FROM iceberg.weather_sync_weather.weather
    WHERE temperature_avg IS NOT NULL
    GROUP BY station_state
    ORDER BY avg_temp DESC
    LIMIT 10
""")

avg_temp_by_state.show()

Average Temperature by State (Top 10):
+--------------+--------+------------+------------+------------+
| station_state|avg_temp|avg_max_temp|avg_min_temp|record_count|
+--------------+--------+------------+------------+------------+
|   Puerto Rico|   81.36|       86.57|       75.62|         106|
|        Hawaii|   76.38|       82.95|       69.37|         530|
|       Florida|   73.93|       82.22|       65.17|        1272|
|     Louisiana|   69.48|       78.93|       59.54|         522|
|         Texas|   68.18|       78.85|       57.02|        2544|
|       Alabama|   66.53|       77.23|       55.29|         424|
|   Mississippi|   66.47|       77.34|       55.16|         742|
|       Arizona|   66.29|       81.01|       51.06|         530|
|       Georgia|   66.22|       77.44|        54.5|         636|
|South Carolina|   65.19|       75.86|       54.05|         424|
+--------------+--------+------------+------------+------------+



In [9]:
# Cities with highest precipitation
print("Cities with Highest Precipitation (Top 10):")
high_precipitation = spark.sql("""
    SELECT 
        station_city,
        station_state,
        ROUND(AVG(precipitation), 2) as avg_precipitation,
        COUNT(*) as measurements
    FROM iceberg.weather_sync_weather.weather
    WHERE precipitation IS NOT NULL AND precipitation > 0
    GROUP BY station_city, station_state
    ORDER BY avg_precipitation DESC
    LIMIT 10
""")

high_precipitation.show(truncate=False)

Cities with Highest Precipitation (Top 10):
+--------------+-------------+-----------------+------------+
|station_city  |station_state|avg_precipitation|measurements|
+--------------+-------------+-----------------+------------+
|Ketchikan     |Alaska       |2.6              |100         |
|Quillayute    |Washington   |2.35             |102         |
|Hilo          |Hawaii       |2.21             |106         |
|Yakutat       |Alaska       |2.2              |100         |
|Baton Rouge   |Louisiana    |1.98             |86          |
|Mt. Washington|New Hampshire|1.93             |18          |
|Astoria       |Oregon       |1.73             |98          |
|Cordova       |Alaska       |1.71             |102         |
|Annette       |Alaska       |1.69             |102         |
|Redding       |California   |1.63             |60          |
+--------------+-------------+-----------------+------------+



In [10]:
# Temperature distribution statistics
print("Temperature Distribution Statistics:")
temp_stats = spark.sql("""
    SELECT 
        COUNT(*) as total_records,
        ROUND(MIN(temperature_min), 2) as coldest_temp,
        ROUND(MAX(temperature_max), 2) as hottest_temp,
        ROUND(AVG(temperature_avg), 2) as overall_avg_temp,
        ROUND(STDDEV(temperature_avg), 2) as temp_std_dev
    FROM iceberg.weather_sync_weather.weather
    WHERE temperature_avg IS NOT NULL
""")

temp_stats.show()

Temperature Distribution Statistics:
+-------------+------------+------------+----------------+------------+
|total_records|coldest_temp|hottest_temp|overall_avg_temp|temp_std_dev|
+-------------+------------+------------+----------------+------------+
|        33486|       -35.0|       111.0|           56.09|        18.8|
+-------------+------------+------------+----------------+------------+



## Step 7: Advanced Analytics with DataFrame API

Use Spark's DataFrame API for more complex transformations.

In [11]:
from pyspark.sql.functions import col, avg, count, round as spark_round

# Monthly temperature trends by year
monthly_trends = weather_df \
    .filter(col("temperature_avg").isNotNull()) \
    .groupBy("date_year", "date_month") \
    .agg(
        spark_round(avg("temperature_avg"), 2).alias("avg_temp"),
        count("*").alias("record_count")
    ) \
    .orderBy("date_year", "date_month")

print("Monthly Temperature Trends:")
monthly_trends.show(20)

Monthly Temperature Trends:
+---------+----------+--------+------------+
|date_year|date_month|avg_temp|record_count|
+---------+----------+--------+------------+
|     2016|         1|   35.37|        3150|
|     2016|         2|   40.87|        2520|
|     2016|         3|   48.67|        2524|
|     2016|         4|   53.89|        2530|
|     2016|         5|   61.14|        3170|
|     2016|         6|   71.56|        2524|
|     2016|         7|   75.11|        3144|
|     2016|         8|   74.41|        2536|
|     2016|         9|   69.41|        2530|
|     2016|        10|   59.56|        3170|
|     2016|        11|   49.76|        2518|
|     2016|        12|   35.65|        2536|
|     2017|         1|   39.77|         634|
+---------+----------+--------+------------+



In [12]:
# Wind speed analysis by state
from pyspark.sql.functions import max as spark_max, min as spark_min

wind_analysis = weather_df \
    .filter(col("wind_speed").isNotNull()) \
    .groupBy("station_state") \
    .agg(
        spark_round(avg("wind_speed"), 2).alias("avg_wind_speed"),
        spark_round(spark_max("wind_speed"), 2).alias("max_wind_speed"),
        spark_round(spark_min("wind_speed"), 2).alias("min_wind_speed")
    ) \
    .orderBy(col("avg_wind_speed").desc())

print("Wind Speed Analysis by State (Top 10):")
wind_analysis.show(10)

Wind Speed Analysis by State (Top 10):
+-------------+--------------+--------------+--------------+
|station_state|avg_wind_speed|max_wind_speed|min_wind_speed|
+-------------+--------------+--------------+--------------+
|New Hampshire|         18.02|          61.1|          1.58|
|       Hawaii|          8.97|         19.22|           1.3|
|Massachusetts|          8.75|         15.84|           0.0|
| South Dakota|          8.47|         16.87|          2.82|
| North Dakota|          8.46|          17.5|           1.4|
|     Oklahoma|          8.46|         16.08|           1.8|
|       Kansas|          8.25|          16.2|          2.05|
|     Nebraska|          8.05|         15.64|          2.62|
|         Iowa|          8.01|         14.33|           2.4|
|        Texas|          7.73|          20.3|          0.55|
+-------------+--------------+--------------+--------------+
only showing top 10 rows



## Step 8: Data Export (Optional)

Convert query results to pandas for visualization.

In [14]:
# Convert to pandas for visualization (only for small datasets)
avg_temp_pandas = avg_temp_by_state.toPandas()
print("Converted to Pandas DataFrame:")
print(avg_temp_pandas)

Converted to Pandas DataFrame:
    station_state  avg_temp  avg_max_temp  avg_min_temp  record_count
0     Puerto Rico     81.36         86.57         75.62           106
1          Hawaii     76.38         82.95         69.37           530
2         Florida     73.93         82.22         65.17          1272
3       Louisiana     69.48         78.93         59.54           522
4           Texas     68.18         78.85         57.02          2544
5         Alabama     66.53         77.23         55.29           424
6     Mississippi     66.47         77.34         55.16           742
7         Arizona     66.29         81.01         51.06           530
8         Georgia     66.22         77.44         54.50           636
9  South Carolina     65.19         75.86         54.05           424


## Summary

This notebook demonstrated:
1. ✅ Loading Spark configuration from external files 
2. ✅ Connecting Spark to Iceberg REST catalog
3. ✅ Querying weather data synced from MySQL via OLake
4. ✅ Performing aggregations and analytics using Spark SQL
5. ✅ Using DataFrame API for complex transformations

### Configuration Architecture

This setup follows:
- Spark: Uses `/opt/spark/conf` for configuration files

In [ ]:
# Optional: Stop the Spark session when completely done
# spark.stop()

In [ ]:
print("🧪 Testing Iceberg Catalog...")

In [ ]:
print("Available catalogs:")
spark.sql("SHOW CATALOGS").show()